# Introduction

This script provides a single, consolidated experimental framework for simulating three social choice algorithms: Plurality, Veto, and Borda.

By combining all functions into one file and using a central execution block, it eliminates code redundancy and provides a robust tool for comparing the performance of different voting rules. 

The script takes user input to select
the desired voting rule and then outputs key metrics, including distortion.

# Import Necessary Libralies

In [1]:
import random
import numpy as np
import collections

In [13]:
# === PART 1: Data Generation Functions (Shared) ===
def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.
    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.
                                 'strong_preference': One alternative has a utility of 1.00, others are 0.00.
                                 'unit_sum': All utilities for a single agent sum to 1.
    """
    utilities = []
    # Check for valid distribution type
    valid_distributions = ['uniform', 'normal', 'strong_preference', 'unit_sum']
    if distribution_type not in valid_distributions:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    # Generate utilities based on the specified distribution
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            # Uniform distribution: each utility value is equally likely to be between 0 and 1.
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            # Normal distribution: values are clustered around a mean.
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities] # clip the values to ensure they stay within the [0, 1] range.
        elif distribution_type == 'strong_preference':
            # Strong Preference: one alternative is a clear favorite (1.00), all others are worthless (0.00).
            agent_utilities = [0.00] * num_alternatives
            strong_choice_index = random.randint(0, num_alternatives - 1)
            agent_utilities[strong_choice_index] = 1.00
        elif distribution_type == 'unit_sum':
            # Unit-Sum: utilities are randomly distributed such that their sum is 1.
            points = sorted([0.0] + [random.uniform(0, 1) for _ in range(num_alternatives - 1)] + [1.0])
            agent_utilities = [points[i+1] - points[i] for i in range(num_alternatives)]
            
        utilities.append(agent_utilities)
    return utilities

def generate_random_preference_list(num_alternatives):
    """Generates a random permutation of alternatives to represent an agent's preferences."""
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """Simulates preferences for a given number of agents and alternatives."""
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Functions (Specific) ===

def plurality_voting(preferences):
    """
    Implements the Plurality voting rule.
    Each agent votes for their most preferred alternative. 
    The alternative with the most votes wins.
    """
    if not preferences:
        return {"winner": None, "votes": 0, "scores": {}}

    vote_counts = collections.Counter(pref[0] for pref in preferences)
    winner = None
    max_votes = -1

    for alternative, votes in vote_counts.items():
        if votes > max_votes:
            max_votes = votes
            winner = alternative

    return {
        "winner": winner,
        "score": max_votes,
        "scores": dict(vote_counts)
    }

def veto_voting(preferences):
    """
    Implements the Veto voting rule.
    Each agent "vetoes" their last-ranked alternative. The alternative with the fewest vetoes wins.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}

    num_alternatives = len(preferences[0])
    veto_scores = {alt: 0 for alt in preferences[0]}

    for preference_list in preferences:
        last_choice = preference_list[num_alternatives - 1]
        veto_scores[last_choice] += 1
    
    winner = None
    min_vetoes = float('inf')

    for alternative, score in veto_scores.items():
        if score < min_vetoes:
            min_vetoes = score
            winner = alternative
    
    # Calculate positive scores for display (optional, but cleaner)
    scores_for_display = {alt: len(preferences) - score for alt, score in veto_scores.items()}

    return {
        "winner": winner,
        "score": len(preferences) - min_vetoes,
        "scores": scores_for_display
    }

def borda_voting(preferences):
    """
    Implements the Borda voting rule.
    Points are assigned based on rank. The alternative with the highest total score wins.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}
    
    num_alternatives = len(preferences[0])
    borda_scores = collections.defaultdict(int)

    # Calculate points for each alternative based on its rank
    for preference_list in preferences:
        for i, alternative in enumerate(preference_list):
            points = num_alternatives - 1 - i
            borda_scores[alternative] += points
    
    winner = None
    max_score = -1

    for alternative, score in borda_scores.items():
        if score > max_score:
            max_score = score
            winner = alternative
    
    return {
        "winner": winner,
        "score": max_score,
        "scores": dict(borda_scores)
    }

# === PART 3: Distortion Calculation Function (Shared) ===
def calculate_distortion(utilities, winner):
    """Calculates the distortion of a voting rule's outcome."""
    if not utilities:
        return { "optimal_alternative": None, "optimal_social_welfare": 0,
            "chosen_alternative": winner, "achieved_social_welfare": 0,
            "distortion": 0}

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion}

# === Main Experimental Framework ===

if __name__ == "__main__":
    # User Input Panel
    voting_rule = str.lower(input("Select voting rule ('plurality', 'veto', or 'borda'): "))
    num_agents = int(input("Enter the number of agents: "))
    num_alternatives = int(input("Enter the number of alternatives: "))
    distribution_type = str.lower(input("Select distribution ('uniform', 'normal', 'strong_preference', or 'unit_sum'): "))

    # Validate the voting rule input
    if voting_rule not in ['plurality', 'veto', 'borda']:
        print("Error: Invalid voting rule. Please choose 'plurality', 'veto', or 'borda'.")
    else:
        # Generate data
        cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
        ordinal_preferences = generate_preferences(num_agents, num_alternatives)
        
        # Select and run the correct voting algorithm
        if voting_rule == 'plurality':
            results = plurality_voting(ordinal_preferences)
        elif voting_rule == 'veto':
            results = veto_voting(ordinal_preferences)
        else: # 'borda'
            results = borda_voting(ordinal_preferences)

        winner = results['winner']
        
        # Calculate distortion
        distortion_results = calculate_distortion(cardinal_utilities, winner)
        
        # Analysis Output Panel
        print(f"\n=== Analysis Results for {voting_rule.capitalize()} Voting ===")
        print(f"  - Parameters:")
        print(f"    - Number of Agents: {num_agents}")
        print(f"    - Number of Alternatives: {num_alternatives}")
        print(f"    - Distribution Type: {distribution_type}")
        
        print("\n=== Analysis Results ===")
        print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
        print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
        print(f"  - Chosen Alternative (by {voting_rule.capitalize()}): {distortion_results['chosen_alternative']}")
        print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
        print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Analysis Results for Plurality Voting ===
  - Parameters:
    - Number of Agents: 10
    - Number of Alternatives: 5
    - Distribution Type: normal

=== Analysis Results ===
  - Optimal Alternative: A4
  - Optimal Social Welfare: 6.56
  - Chosen Alternative (by Plurality): A3
  - Achieved Social Welfare: 5.20
  - Distortion: 1.26
